In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pandas_profiling
from pandas.plotting import parallel_coordinates
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
import sklearn



#Environment Settings 

np.set_printoptions(precision=4)                    # To display values only upto four decimal places. 
pd.set_option('mode.chained_assignment', None)      # To suppress pandas warnings.
pd.set_option('display.max_colwidth', -1)           # To display all the data in each column
pd.options.display.max_columns = 50                 # To display every column of the dataset in head()
plt.style.use('seaborn-whitegrid')                  # To apply seaborn whitegrid style to the plots.
%matplotlib inline
sns.set(style='whitegrid', font_scale=1.3, color_codes=True)      # To adjust seaborn settings for the plots.
warnings.filterwarnings('ignore')                   # To suppress all the warnings in the notebook.


# Import Dataset

In [ ]:
#df_KPI = pd.read_csv("C:\\Users\\esauson\\Desktop\\BMAS Hackathon\\7_NPI_KPS_Corelation\\kpi_data_to_share_train.csv")
#df_NPS = pd.read_csv("C:\\Users\\esauson\\Desktop\\BMAS Hackathon\\7_NPI_KPS_Corelation\\nps_data_to_share_train.csv")

df_raw_input = pd.read_csv("C:\\Users\\esauson\\Desktop\\BMAS Hackathon\\7_NPI_KPS_Corelation\\kpi_data_to_share_test_raw.csv")

In [ ]:
print(df_KPI.shape)
print(df_NPS.shape)
print(df_raw_input.shape)

df_raw_input.columns

In [ ]:
X_Predict = df_raw_input[['region', 'cluster', 'month', 'cell_name', 'kpi2', 'kpi4','kpi5','kpi6', 'kpi7']]

X_Predict.shape


In [ ]:
df_KPI.info()
df_NPS.info()
df_NPS.info()
df_NPS.isnull().sum()# Check for count of null values


In [ ]:
df_merged = pd.merge(df_KPI,df_NPS,how='outer')
df_merged.shape
df_merged


In [ ]:
df_merged

In [ ]:
df_merged.isna().sum() #check the missing values or null values

In [ ]:
plt.figure(figsize=(14,14))

corr = df_merged.corr()

sns.heatmap(corr,annot=True,square=True,cmap='BuPu')

# Drop columns kpi1,kpi3,kpi 8,kpi9,start_time


In [ ]:
df_merged.drop(['kpi1','kpi3','kpi8','kpi9','start_time'],axis=1,inplace=True)

df_merged.shape

In [ ]:
sns.distplot(df_merged.nps_values, color="b") # to check the skewness of target variable

In [ ]:
#definng x and y 

y = df_merged['nps_values']
x = df_merged[['region', 'cluster', 'month', 'cell_name', 'kpi2', 'kpi4','kpi5','kpi6', 'kpi7']]
y.head(10)
x.head(10)

# Splitting the data set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test=train_test_split(x,y, test_size=0.20, random_state=1)

print ("\nXtrain values are\n",X_train.shape)
print ("\nXtest values are\n",X_test.shape)
print ("\nYtrain values are\n",Y_train.shape)
print ("\nYtest values are\n",Y_test.shape)


# Linear Regression Model

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, Y_train) 

NPS_pred_test_Linear_Reg = linreg.predict(X_test)
NPS_pred_train_Linear_Reg = linreg.predict(X_train)



# Model Evaluation

In [ ]:
#RMSE_test = np.sqrt(metrics.mean_squared_error(Y_test, NPS_pred_test_Linear_Reg))
RMSE_train = np.sqrt(metrics.mean_squared_error(Y_train, NPS_pred_train_Linear_Reg))

#RMSE_test

RMSE_train

In [ ]:
Yhat = linreg.predict(X_train)
SS_Residual = sum((Y_train-Yhat)**2)
SS_Total = sum((Y_train-np.mean(Y_train))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(Y_train)-1)/(len(Y_train)-X_train.shape[1]-1)
print(r_squared, adjusted_r_squared)

# Logistic Regression

In [ ]:
df_merged.shape

In [ ]:
df_merged.head()

In [ ]:
df_merged['Target_Var'] = [1 if x <= 0.1 else 0 for x in df_merged['nps_values']]

df_merged['Target_Var'].unique()


In [ ]:
df_merged.shape

df_merged.columns

df_merged.info()

In [ ]:
plt.figure(figsize=(11,11))

corr = df_merged.corr()

sns.heatmap(corr,annot=True,square=True)

In [ ]:
y = df_merged['Target_Var']
x = df_merged[['region', 'cluster', 'month', 'cell_name', 'kpi2', 'kpi4','kpi5','kpi6', 'kpi7']]
y.head(10)
x.head(10)

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
#scaler = MinMaxScaler().fit(x)
scaler = RobustScaler().fit(x)
X_scaled = pd.DataFrame(scaler.transform(x),columns=['region', 'cluster', 'month', 'cell_name', 'kpi2', 'kpi4','kpi5','kpi6', 'kpi7'])
X_scaled.head()

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(x,y, test_size=0.20, random_state=1)

print ("\nXtrain values are\n",X_train.shape)
print ("\nXtest values are\n",X_test.shape)
print ("\nYtrain values are\n",Y_train.shape)
print ("\nYtest values are\n",Y_test.shape)


In [ ]:
#Logistic model 

logreg = LogisticRegression()
logreg.fit(X_train,Y_train)

In [ ]:
# make predictions
predicted_Logistic = logreg.predict(X_test)
#confusion metrix
metrics.confusion_matrix(Y_test,predicted_Logistic)

In [ ]:
y_pred_train = logreg.predict(X_train)

y_pred_test = logreg.predict(X_test)

y_pred_prob_test = logreg.predict_proba(X_test)
#y_pred_prob_train = logreg.predict_proba(X_train)

y_pred_train, y_pred_test, y_pred_prob_test

In [ ]:
AR = metrics.accuracy_score(Y_test,y_pred_test)

AR

In [ ]:
from sklearn.metrics import confusion_matrix

CF = pd.DataFrame(confusion_matrix(Y_test,y_pred_test))
CF.index = ['Actual_Good NPS',"Actual_Poor NPS"]
CF.columns = ['PR_Good NPS',"PR_Poor NPS"]

print(CF)

In [ ]:
Y_test.shape

# DECISION TREE 

In [ ]:
dt_model = DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [ ]:
dt_model.fit(X_train, Y_train)

In [ ]:
dt_model.score(X_train , Y_train)

In [ ]:
dt_model.score(X_test , Y_test)

In [ ]:
# make predictions
predicted_DT = dt_model.predict(X_test)
#confusion metrix
metrics.confusion_matrix(predicted_DT,Y_test)

In [ ]:
predicted_DT

# Regularise Decision Tree

In [ ]:
dt_model = DecisionTreeClassifier(max_depth = 11, min_samples_leaf=10, min_samples_split=5 )
dt_model.fit(X_train, Y_train)

In [ ]:
print("Train Data Accuracy score: ")
dt_model.score(X_train , Y_train)

In [ ]:
print("Test Data Accuracy score: ")
dt_model.score(X_test , Y_test)

In [ ]:
# make predictions
predicted_DT_Regularise = dt_model.predict(X_test)
#confusion metrix
metrics.confusion_matrix(predicted_DT_Regularise,Y_test)

# Random Forest Algorithm

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfcl = RandomForestClassifier(n_estimators = 100, max_depth=3, max_features=0.20, max_leaf_nodes=20)
rfcl = rfcl.fit(X_train, Y_train)

In [ ]:
print("Test Data Accuracy score: ")
rfcl.score(X_test , Y_test)

In [ ]:
print("Train Data Accuracy score: ")
rfcl.score(X_train , Y_train)

In [ ]:
# make predictions
predicted_RF = rfcl.predict(X_test)
#confusion metrix
metrics.confusion_matrix(predicted_RF,Y_test)

# Adaboost Ensemble Algorithm

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abcl = AdaBoostClassifier(base_estimator=dt_model, n_estimators=5)
abcl = abcl.fit(X_train, Y_train)

In [ ]:
print("Train Data Accuracy score: ", abcl.score(X_train, Y_train))


print("Test Data Accuracy score: ",abcl.score(X_test, Y_test))


In [ ]:
# make predictions
predicted_ADA = abcl.predict(X_test)
#confusion metrix
metrics.confusion_matrix(predicted_ADA,Y_test)

# Bagging Classifier Algorithm

In [ ]:
from sklearn.ensemble import BaggingClassifier

bgcl = BaggingClassifier(base_estimator=dt_model, n_estimators=20)
bgcl.fit(X_train, Y_train)

In [ ]:
print("Train Data Accuracy score: ")
bgcl.score(X_train, Y_train)


In [ ]:

print("Test Data Accuracy score: ")
bgcl.score(X_test, Y_test)

In [ ]:
# make predictions
predicted_BAG = bgcl.predict(X_test)
#confusion metrix
metrics.confusion_matrix(predicted_BAG,Y_test)

#predicted_BAG

# GradientBoost Classifier Algorithm# 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbcl = GradientBoostingClassifier(n_estimators = 50, learning_rate = 0.09, max_depth=11)
gbcl.fit(X_train, Y_train)

In [ ]:
print("Train Data Accuracy score: ")
gbcl.score(X_train, Y_train)

In [ ]:
print("Test Data Accuracy score: ")
gbcl.score(X_test, Y_test)

In [ ]:
# make predictions
predicted_GB = gbcl.predict(X_test)
#confusion metrix
metrics.confusion_matrix(predicted_GB,Y_test)

In [ ]:
predict_dict = {'predicted_DT_Regularise':predicted_DT_Regularise,'predicted_RF':predicted_RF,'predicted_ADA':predicted_ADA,'predicted_BAG':predicted_BAG,'predicted_Logistic': predicted_Logistic}
predict_dict 

In [ ]:
df_predictions = pd.DataFrame(predict_dict)
df_predictions.head()

In [ ]:
df_predictions['Vote'] = df_predictions['predicted_ADA'] + df_predictions['predicted_BAG'] + df_predictions['predicted_DT_Regularise'] + df_predictions['predicted_RF'] + df_predictions['predicted_Logistic']

df_predictions

In [ ]:
df_predictions['Predicted TV'] = [1 if x >=3 else 0 for x in df_predictions['Vote']] 

In [ ]:
df_predictions['Actual TV'] = Y_test.values 

df_predictions.head(10)

#df_predictions.shape



In [ ]:
# Result to Excel
from pandas import ExcelWriter

writer = csWriter('NPS_KPI_Corelation_Result.xlsx')
df_predictions.to_excel(writer,'NPS_KPI_Corelatuion_Result')
writer.save() 
 


In [ ]:
from sklearn.metrics import confusion_matrix 

CF_model_prediction_result = pd.DataFrame(confusion_matrix(df_predictions['Predicted TV'],df_predictions['Actual TV']))

CF_model_prediction_result.index = ["Model_Negative","Models_Positive",]

CF_model_prediction_result.columns = ['Predict Negative','Predict_Positive',]

print(CF_model_prediction_result) 


In [ ]:
X_Predict.shape

In [ ]:
DT_OP = dt_model.predict(X_Predict)
LR_OP = logreg.predict(X_Predict)
RF_OP = rfcl.predict(X_Predict)
ADA_OP = abcl.predict(X_Predict)
BAG_OP = bgcl.predict(X_Predict)


In [ ]:
d1 = {'DT_OP':DT_OP, 'LR_OP':LR_OP,'RF_OP':RF_OP,'ADA_OP':ADA_OP,'BAG_OP':BAG_OP}

In [ ]:
df_predict_raw = pd.DataFrame(d1)

In [ ]:
df_predict_raw.shape

In [ ]:
from pandas import ExcelWriter

writer = ExcelWriter('NPS_KPI_Corelation_Result.xlsx')
df_predictions.to_excel(writer,'NPS_KPI_Corelatuion_Result')
writer.save() 

In [ ]:
df_predict_raw.to_csv("NPS_KPI.csv")

In [ ]:
from sklearn.metrics import roc_curve

sklearn.metrics.roc_curve(y_true, y_score, pos_label=None, sample_weight=None, drop_intermediate=True)